# Охват иммунизацией от полиомиелита (Pol3) среди детей в возрасте 1 года (%)

In [1]:
import pandas as pd

from helpers import (
    get_data_with_full_locations_and_years, 
    get_omissions, 
)

from warnings import simplefilter
simplefilter('ignore')

Эти данные можно найти на сайте ВОЗ по ссылке

https://www.who.int/data/gho/data/indicators/indicator-details/GHO/polio-(pol3)-immunization-coverage-among-1-year-olds-(-)

In [2]:
# Загрузим таблицу
data = pd.read_csv('../data/who_polio_immunization.csv')

data.head(2)

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WHS4_544,Polio (Pol3) immunization coverage among 1-yea...,numeric,WPR,Western Pacific,Country,PNG,Papua New Guinea,Year,2020,...,NaN,NaN,NaN,NaN,NaN,43,NaN,NaN,EN,2024-07-11T21:00:00.000Z
1,WHS4_544,Polio (Pol3) immunization coverage among 1-yea...,numeric,AFR,Africa,Country,CAF,Central African Republic,Year,2020,...,NaN,NaN,NaN,NaN,NaN,46,NaN,NaN,EN,2024-07-11T21:00:00.000Z


Описание нужных нам столбцов.

- `ParentLocationCode` - код региона
- `SpatialDimValueCode` - код страны
- `Period` - год
- `FactValueNumeric` - охват иммунизацией от полиомиелита (Pol3) среди детей в возрасте 1 года (%)

Оставим только их.

In [3]:
data = data[[
    'ParentLocationCode', 'SpatialDimValueCode', 'Period', 'FactValueNumeric',
]]

# Переименуем столбец FactValueNumeric в PolioImmunization
data.rename(columns={'FactValueNumeric': 'PolioImmunization'}, inplace=True)

data.head()

,ParentLocationCode,SpatialDimValueCode,Period,PolioImmunization
0,WPR,PNG,2020,43
1,AFR,CAF,2020,46
2,EMR,SOM,2020,47
3,AFR,GIN,2020,48
4,AFR,AGO,2020,51


В данной таблице для некоторых стран данные заполнены не за все года.\
Добавим строки с отсутствующими годами, чтобы явно увидеть пропуски.

In [4]:
data = get_data_with_full_locations_and_years(
    data,
    {
        'region_code': 'ParentLocationCode',
        'location_code': 'SpatialDimValueCode',
        'year': 'Period',
    }
)

# Посмотрим на количество пропусков
data.isna().sum()

ParentLocationCode      0
SpatialDimValueCode     0
Period                  0
PolioImmunization      19
dtype: int64

In [5]:
# Посмотрим на пропущенные данные
omissions_data = get_omissions(data)

omissions_data.sort_values(by=['SpatialDimValueCode', 'Period'])

,ParentLocationCode,SpatialDimValueCode,Period,PolioImmunization
3965,EUR,MNE,2000,NaN
3770,EUR,MNE,2001,NaN
3575,EUR,MNE,2002,NaN
3380,EUR,MNE,2003,NaN
3185,EUR,MNE,2004,NaN
2990,EUR,MNE,2005,NaN
3919,AFR,SSD,2000,NaN
3724,AFR,SSD,2001,NaN
3529,AFR,SSD,2002,NaN
3334,AFR,SSD,2003,NaN


Заполним их средним по региону.

In [6]:
data['PolioImmunization'] = \
    data.groupby(['ParentLocationCode', 'Period'])['PolioImmunization'] \
    .transform(lambda x: x.fillna(x.mean()));

# Убедимся, что пропусков нет
data.isna().sum().sum()

0

In [7]:
# Удалим столбцы, которые больше не нужны
data.drop(columns=['ParentLocationCode'], inplace=True)

In [8]:
# Сохраним полученную таблицу для дальнейшего использования
data.to_csv('../data/who_polio_immunization_prepared.csv', index=False)